# 02-01 : Use Remote Docker Instance

Attempt to use the MLRun deployed with `mlsetup.py` to run a remote docker instance.

In [ ]:
import os

In [ ]:
import logging

# configure log format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s [%(name)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# get the logger
logger = logging.getLogger(__name__)

## 1. Configuration 

In [ ]:
# set the api url
mlrun_api_server = 'dragon'
mlrun_api_port = 8080
os.environ["MLRUN_DBPATH"] = f"http://{mlrun_api_server}:{mlrun_api_port}"

# load the mlrun module
import mlrun

In [ ]:
MLRUN_PROJECT = "use-remote"
MLRUN_DOCKER_IMAGE = "mlrun/mlrun"

## 2. Set Project

In [ ]:
project = mlrun.get_or_create_project(
    name=MLRUN_PROJECT,
    context='./',) 
    
project.save()

## 3. Create Function

### 3.1 Create Function Code File

In [ ]:
%%writefile hello_world.py

import mlrun

# get the logger
import logging
logger = logging.getLogger(__name__)

def handler(context: mlrun.MLClientCtx = None):
    logger.info("Hello world")

    context.log_result("hello", "world")

    return "Hello world result!"

### 3.2 Create Project Function

In [ ]:
hello_world = mlrun.code_to_function(
    name='hello-world',
    project=MLRUN_PROJECT,
    filename='hello_world.py',
    handler='handler',
    kind='job',
    image=MLRUN_DOCKER_IMAGE,
    description='Hello world function',
)
hello_world.save()

In [ ]:
project.build_function(hello_world)

## 4. Run Function

In [ ]:
run = hello_world.run(function=hello_world,
                      handler='handler',
                      image=MLRUN_DOCKER_IMAGE,
                      watch=True)